## Experimental notebook

This is a notebook where we experimented with different feature combinations. It's not part of the competition submission.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

def get_score(actuals, preds):
    new_test= pd.DataFrame({'Actuals': actuals,'Preds': preds})
    new_test = new_test.loc[new_test['Actuals'] != 0,:]
    return metric(np.array(new_test['Actuals']), np.array(new_test['Preds']))

def mean_encoder(df, col, target = 'Sales'):
    Mean_encoded_subject = df.groupby([col])[target].mean().to_dict() 
    return df[col].map(Mean_encoded_subject)

## Dirty Data Imports (don't run)

In [3]:
# train = pd.read_csv('data/train.csv', parse_dates = True)
# stores = pd.read_csv('data/store.csv')

## Clean Data imports (run these!)

In [4]:
no_null_sales = pd.read_csv('clean_data.csv', parse_dates = True)
stores = pd.read_csv('stores_light.csv')

/opt/anaconda3/envs/minicomp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Base cleaning (don't run)

In [5]:
# fillna_storetype_a = stores.groupby('StoreType').median().loc['a', 'CompetitionDistance']
# fillna_storetype_b = stores.groupby('StoreType').median().loc['b', 'CompetitionDistance']
# fillna_storetype_c = stores.groupby('StoreType').median().loc['c', 'CompetitionDistance']
# fillna_storetype_d = stores.groupby('StoreType').median().loc['d', 'CompetitionDistance']
# stores.loc[(stores['CompetitionDistance'].isnull()) & (stores.loc[:,'StoreType'] == 'a')] = fillna_storetype_a
# stores.loc[(stores['CompetitionDistance'].isnull()) & (stores.loc[:,'StoreType'] == 'b')] = fillna_storetype_b
# stores.loc[(stores['CompetitionDistance'].isnull()) & (stores.loc[:,'StoreType'] == 'c')] = fillna_storetype_c
# stores.loc[(stores['CompetitionDistance'].isnull()) & (stores.loc[:,'StoreType'] == 'd')] = fillna_storetype_d

# train.loc[(train.loc[:, 'Sales'].isnull()) & (train['Open'] == 0), 'Sales'] = 0
# train.loc[(train.loc[:, 'Sales'].isnull()) & (train['Customers'] == 0), 'Sales'] = 0

# train.loc[(train.loc[:, 'Customers'].isnull()) & (train['Sales'] == 0), 'Customers'] = 0

# no_null_sales = train.loc[(train['Sales'].notnull()) | (train['Open'] == 0), :]
# no_null_sales.loc[:,'Sales'].fillna(0, inplace=True)

# no_null_sales = no_null_sales.loc[no_null_sales['Store'].notnull(), :]

# no_null_sales.loc[:,'Date'] = pd.to_datetime(no_null_sales.loc[:,'Date'], format='%Y-%m-%d')
# no_null_sales.loc[:,'DayOfWeek'] = no_null_sales.loc[:,'Date'].dt.weekday + 1

# no_null_sales.loc[no_null_sales['StateHoliday'] == 0.0, 'StateHoliday'] = '0'
# no_null_sales.loc[no_null_sales['StateHoliday'] == 0, 'StateHoliday'] = '0'

# no_null_sales.loc[(no_null_sales['Open'].isnull()) & (no_null_sales['Sales'] == 0),'Open'] = 0
# no_null_sales.loc[(no_null_sales['Open'].isnull()) & (no_null_sales['Sales'] > 0),'Open'] = 1

## Baseline 1

In [6]:
X = no_null_sales[['Store']]
y = no_null_sales['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
lazy_est = pd.concat([X_train, y_train], axis=1)
lazy_est = lazy_est.groupby(['Store']).mean()
preds = X_test.merge(lazy_est, how = 'left', on = 'Store').loc[:, 'Sales']
actuals = np.array(y_test)
get_score(actuals, preds)

40.58064127185794

## Baseline 2

In [7]:
X = no_null_sales[['Store', 'DayOfWeek']]
y = no_null_sales['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
lazy_est = pd.concat([X_train, y_train], axis=1)
lazy_est = lazy_est.groupby(['Store', 'DayOfWeek']).mean()
preds = X_test.merge(lazy_est, how = 'left', on = ['Store', 'DayOfWeek']).loc[:, 'Sales']
actuals = np.array(y_test)
get_score(actuals, preds)

25.42650037519698

## Baseline 3

In [8]:
X = no_null_sales[['Store', 'DayOfWeek', 'Open']]
y = no_null_sales['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
lazy_est = pd.concat([X_train, y_train], axis=1)
lazy_est = lazy_est.groupby(['Store', 'DayOfWeek', 'Open']).mean()
preds = X_test.merge(lazy_est, how = 'left', on = ['Store', 'DayOfWeek', 'Open']).loc[:, 'Sales']
actuals = np.array(y_test)
get_score(actuals, preds)

23.438336545019375

## Further cleaning (don't run unless you're sure!)

In [9]:
#WILL TAKE 30 MINS TO RUN!!!!!!
#Slow and ugly way to fill in NaNs with the mode
holiday_nulls = no_null_sales.loc[no_null_sales['StateHoliday'].isnull()]
for i, row in holiday_nulls.iterrows():
    date = str(row['Date'])
    mode = no_null_sales.loc[no_null_sales['Date'] == date, 'StateHoliday'].mode()[0]
    no_null_sales.loc[i, 'StateHoliday'] = mode
    
holiday_nulls = no_null_sales.loc[no_null_sales['SchoolHoliday'].isnull()]
for i, row in holiday_nulls.iterrows():
    date = str(row['Date'])
    mode = no_null_sales.loc[no_null_sales['Date'] == date, 'SchoolHoliday'].mode()[0]
    no_null_sales.loc[i, 'SchoolHoliday'] = mode

In [10]:
customer_means = no_null_sales.groupby(['Store', 'DayOfWeek']).mean()['Customers']

customer_nulls = no_null_sales.loc[no_null_sales['Customers'].isnull()]
for i, row in customer_nulls.iterrows():
    store = row['Store']
    DayOfWeek = row['DayOfWeek']
    mean = customer_means.loc[store, DayOfWeek]
    no_null_sales.loc[i, 'Customers'] = mean

In [11]:
no_null_sales.reset_index(inplace = True, drop = True)

null_promo = no_null_sales.loc[no_null_sales['Promo'].isnull()]

for i, row in null_promo.iterrows():
    above_and_below = no_null_sales.loc[i-1, 'Promo'] + no_null_sales.loc[i+1, 'Promo']
    if above_and_below == 2:
        no_null_sales.loc[i, 'Promo'] = 1
    elif above_and_below == 0:
        no_null_sales.loc[i, 'Promo'] = 0
        
null_promo = no_null_sales.loc[no_null_sales['Promo'].isnull()]

for i, row in null_promo.iterrows():
    date = str(row['Date'])
    mean = no_null_sales.groupby('Date').mean().loc[date, 'Promo']
    if mean > 0.9:
        no_null_sales.loc[i, 'Promo'] = 1
    else:
        no_null_sales.loc[i, 'Promo'] = 0

## Random Forest 1st attempt

In [12]:
rf_set = no_null_sales.drop(['Date', 'Customers', 'SchoolHoliday', 'Promo'], axis = 1)
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
get_score(actuals, preds)

23.732235548760748

## 20 % sample experiments

In [13]:
sample = no_null_sales.sample(frac = 0.2, random_state = 42)

In [14]:
# Without promo

rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Promo'], axis = 1)
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
get_score(actuals, preds)

25.443833860208215

In [15]:
#With Promo

rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday'], axis = 1)
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
get_score(actuals, preds)

19.58039458157083

In [16]:
##With School holiday

rf_set = sample.drop(['Date', 'Customers'], axis = 1)
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
get_score(actuals, preds)

19.823696569594205

In [21]:
sample['Date'] = pd.to_datetime(sample['Date'])

In [22]:
sample['Month'] = sample['Date'].dt.month
sample['Week'] = sample['Date'].dt.week

<ipython-input-22-fbdd7e3ba0cf>:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  sample['Week'] = sample['Date'].dt.week


In [23]:
#With Month (tried both with and without mean encoder, neither was good)
rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Week'], axis = 1)
rf_set['Month'] = mean_encoder(rf_set, 'Month')
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
get_score(actuals, preds)

21.639495155361388

In [24]:
#With Week (tried both with and without mean encoder, neither was good)
rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Month'], axis = 1)
rf_set['Week'] = mean_encoder(rf_set, 'Week')
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
get_score(actuals, preds)

24.62553299664485

In [25]:
rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Week'], axis = 1)
rf_set.loc[rf_set['Month'] != 12, 'Month'] = 0
rf_set.loc[rf_set['Month'] == 12, 'Month'] = 1
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
get_score(actuals, preds)

20.07700260504653

In [26]:
#Store mean-encoded

rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Month', 'Week'], axis = 1)
rf_set['Store'] = mean_encoder(rf_set, 'Store')
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
get_score(actuals, preds)

19.557116160542776

In [27]:
#State-Holiday mean-encoded

rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Month', 'Week'], axis = 1)
rf_set['StateHoliday'] = mean_encoder(rf_set, 'StateHoliday')
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
get_score(actuals, preds)

19.551302083751867

In [28]:
#Weekday mean-encoded

rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Month', 'Week'], axis = 1)
rf_set['StateHoliday'] = mean_encoder(rf_set, 'StateHoliday')
rf_set['DayOfWeek'] = mean_encoder(rf_set, 'DayOfWeek')
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
get_score(actuals, preds)

19.551640670899317

In [29]:
# # DON'T RUN!!
## Making dayBeforeHoliday feature, didn't help 

# aDay = datetime.timedelta(days = 1)

# sample2 = no_null_sales.copy()
# sample2.loc[:,'StateHoliday'] = sample2.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
# mapping = sample2.groupby(['Date']).mean()['StateHoliday']
# sample2.loc[:,'DayBeforeHoliday'] = 0
# for i, row in sample2.iterrows():
#     if i == 0:
#         print('Hi!')
#     else:
#         date1 = str(row['Date'])
#         date2 = str(row['Date'] + aDay)
#         if mapping.loc[date2] > 0.5 and mapping.loc[date1] == 0:
#             sample2.loc[i, 'DayBeforeHoliday'] = 1

In [32]:
# #With DayBeforeHoliday

# with_dbh = sample2.sample(frac = 0.2, random_state = 42)

# rf_set = with_dbh.drop(['Date', 'Customers', 'SchoolHoliday'], axis = 1)
# rf_set['StateHoliday'] = mean_encoder(rf_set, 'StateHoliday')
# X = rf_set.drop('Sales', axis = 1)
# y = rf_set['Sales']
# from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor()
# rf.fit(X_train, y_train)
# preds = rf.predict(X_test)
# actuals = np.array(y_test)
# get_score(actuals, preds)

In [31]:
#Date mean-encoded

rf_set = sample.drop(['Customers', 'SchoolHoliday', 'Month', 'Week'], axis = 1)
rf_set['StateHoliday'] = mean_encoder(rf_set, 'StateHoliday')
rf_set['Date'] = mean_encoder(rf_set, 'Date')
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
get_score(actuals, preds)

21.88863399134336